<a href="https://colab.research.google.com/github/hrushikute/DataAnalytics/blob/master/Imdb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf

print(tf.__version__)

2.8.2


In [2]:
import tensorflow_datasets as tfds
imdb, info  = tfds.load("imdb_reviews",with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE5C8SL/imdb_reviews-train.tfrecord


  0%|          | 0/25000 [00:00<?, ? examples/s]

...

  0%|          | 0/25000 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteE5C8SL/imdb_reviews-unsupervised.tfrecord


  0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
import numpy as np

train_data, test_data = imdb['train'],imdb['test']

In [4]:
training_sentences = []
training_labels = []

testing_sentences = []
testing_labels = []

# Loop over all the training example and get the review and labels
for s,l in train_data:
  training_sentences.append(s.numpy().decode('utf8'))
  training_labels.append(l.numpy())

# Loop over all the training example and get the review and labels
for s,l in test_data:
  testing_sentences.append(s.numpy().decode('utf8'))
  testing_labels.append(l.numpy())

In [5]:
# Convert the training labels inot numpy aaray

training_labels_final = np.array(training_labels )
testing_labels_final = np.array(testing_labels )

In [16]:
# Toeknize and padd the training sentences.
vocab_size=10000
embedding_dim =16
max_length = 120
truc_type = 'post'
oov_tok = "<OOV>"

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
wordindex = tokenizer.word_index
sequence = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequence,maxlen=max_length, truncating=truc_type)

test_sequence = tokenizer.texts_to_sequences(testing_sentences)
test_paded = pad_sequences(test_sequence,maxlen=max_length, truncating=truc_type)

In [17]:
# Lets create a model 

model = tf.keras.Sequential(
    [
     tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
     tf.keras.layers.Flatten(),
     tf.keras.layers.Dense(6,activation='relu'),
     tf.keras.layers.Dense(1,activation='sigmoid')
    ]
)

In [18]:
# Compile the model
model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 120, 16)           160000    
                                                                 
 flatten_1 (Flatten)         (None, 1920)              0         
                                                                 
 dense_2 (Dense)             (None, 6)                 11526     
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Fit the model and check the metrics

model.fit(padded,
          training_labels_final,
          epochs=10,
          validation_data=(test_paded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 5s 5ms/step - loss: 0.4852 - accuracy: 0.7524 - val_loss: 0.3734 - val_accuracy: 0.8332
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2509 - accuracy: 0.9030 - val_loss: 0.3938 - val_accuracy: 0.8286
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.1162 - accuracy: 0.9659 - val_loss: 0.4796 - val_accuracy: 0.8134
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0338 - accuracy: 0.9952 - val_loss: 0.5748 - val_accuracy: 0.8137
Epoch 5/10
782/782 [==============================] - 5s 6ms/step - loss: 0.0103 - accuracy: 0.9989 - val_loss: 0.6459 - val_accuracy: 0.8123
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0037 - accuracy: 0.9997 - val_loss: 0.7195 - val_accuracy: 0.8106
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0012 - accuracy: 1.0000 - val_loss: 0.7774 - val_accuracy: 0.8115
Epoch 

In [10]:
# Visualize the word  embeddings

# getting embedding layer from the model ie the first layer
embedding_layer=model.layers[0]

# Get the weights of the embedding layer.
embedding_weights=embedding_layer.get_weights()[0]

print(embedding_weights.shape)


(100000, 16)


In [14]:
# Get teh index-word dictonary
reverse_word_index = tokenizer.index_word


**Need to generate two files:**


1.   vecs.tsv - contains the vector weights of each word in the vocabulary
2.   meta.tsv - contains the words in the vocabulary




In [20]:
import io

out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')

for word_num in range(1, vocab_size):
  #get the word associated with current index.
  word_name=reverse_word_index[word_num]
  # Get embedding weights
  word_embedding = embedding_weights[word_num]
  # Write the word name 
  out_m.write(word_name+"\n")
  #Write the word embeddings
  out_v.write('\t'.join([str(x) for x in word_embedding])+"\n")


out_v.close()
out_m.close()



In [21]:
# Import file utilities in colab
try :
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')
  

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>